# Analiza Wielowymiarowa - zajecia 6 - Analiza głównych składowych (PCA)

In [ ]:
from multidim.utils import resolve_stata, load_stata

STATA_PATH, STATA_TYPE = resolve_stata(version = 17, stype = "se")
# make sure they are proper ones
STATA_PATH, STATA_TYPE

In [ ]:
load_stata(STATA_PATH, STATA_TYPE)

In [ ]:
# Załadowanie bibliotek
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from scipy.linalg import svd

from sklearn import datasets

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


from multidim.funs import corr_mat

In [ ]:
np.random.seed(1234)

## Przykład 1 - Dekompozycja głównych składowych (SVD) a wartości własne (Eigenvalues) and Internals

In [ ]:
iris_set = datasets.load_iris()
iris_x = iris_set["data"]
# CENTRING
iris_x_centred = iris_set["data"] - np.mean(iris_set["data"], axis = 0)
#SCALING
iris_x_scaled = (iris_set["data"] - np.mean(iris_set["data"], axis = 0))/np.std(iris_set["data"], axis = 0)

In [ ]:
# almost always centring
# in most scenerios full scaling
# correlation matrix = scaling

In [ ]:
# when X CENTRED -> X`X/n = COV(X)
np.allclose((iris_x_centred.T @ iris_x_centred) / (iris_x_centred.shape[0] - 1), np.cov(iris_x_centred.T, ddof = 1))

In [ ]:
# when X SCALED -> Corr(X) = COV(X)
np.allclose(np.corrcoef(iris_x_scaled.T), np.cov(iris_x_scaled.T, ddof=0))

In [ ]:
# SVD -> X = U * S * Vt
u, s, vh = np.linalg.svd(iris_x_scaled, full_matrices = False)
np.allclose(iris_x_scaled, u @ np.diag(s) @ vh)

In [ ]:
# Eigen -> C = V * S * Vt
s_eigen, v_eigen = np.linalg.eigh(np.corrcoef(iris_x.T))
np.allclose(np.corrcoef(iris_x.T), v_eigen @ np.diag(s_eigen) @ v_eigen.T)

In [ ]:
# Unit disc
# https://en.wikipedia.org/wiki/Singular_value_decomposition#/media/File:Singular-Value-Decomposition.svg
# SVD could be run for square or long shape
corr_iris_x = np.corrcoef(iris_x.T)
u_s, s_s, vh_s = np.linalg.svd(corr_iris_x, full_matrices = False)
assert np.allclose(
    np.diag(np.ones((4))) @ corr_iris_x, 
    corr_iris_x
)
assert np.allclose(
    np.diag(np.ones((4))) @ u_s @ np.diag(s_s) @ vh_s, 
    corr_iris_x
)

In [ ]:
# principal components
pca_svd = pd.DataFrame(iris_x_scaled @ vh.T)
pca_svd_alt = pd.DataFrame(u @ np.diag(s))
assert np.allclose(pca_svd, pca_svd_alt)
pca_eigen = pd.DataFrame(iris_x_scaled @ v_eigen)

In [ ]:
# direction is stable (kierunek) but not sense (zwrot)
# from sklearn.utils.extmath import svd_flip
# Sign correction to ensure deterministic output from SVD.
# Adjusts the columns of u and the rows of v such that the loadings in the
# columns in u that are largest in absolute value are always positive.

Ładunki czynnikowe (ang. Loadings )

Korelacje między oryginalnymi zmiennymi a składowymi głównymi (correlations between the original variables and the principal components)

In [ ]:
pca_nams = ['PC1', 'PC2', 'PC3', 'PC4']
feature_nams = iris_set["feature_names"]
# loadings from sklearn
pca = PCA(n_components = 4)
X = pca.fit_transform(iris_x_scaled)
loadings1 = pd.DataFrame(pca.components_.T * np.sqrt(pca.explained_variance_), index = pca_nams, columns = feature_nams)
# Loadings from numpy svd decomposition
loadings2 = pd.DataFrame(vh.T * s/np.sqrt(iris_x_scaled.shape[0] - 1), index = pca_nams, columns = feature_nams)
# Loadings by directly getting correlations between variables and prinicpal components
loadings3 = pd.DataFrame(corr_mat(pca_svd, iris_x_scaled).values, columns = pca_nams, index = feature_nams)
loadings1.T, loadings2.T, loadings3

In [ ]:
# e.g. first PCA component == original data * first rotation
assert np.allclose(
    np.round(pca_svd.iloc[:,0].values, 2), 
    np.round(vh.T[:, 0] @ iris_x_scaled.T, 2)
)
ev = pd.DataFrame(vh.T)
ev.index = iris_set.feature_names
ev

Korelacje pomiędzy czynnikami głównymi (ang. correlation between principal components)

In [ ]:
pca_svd.corr(), pca_eigen.corr()

Wartości własne (ang. Eigenvalues)

In [ ]:
index_eigen = [i[0] for i in sorted(enumerate(s_eigen), reverse=True, key = lambda x: x[1])]

In [ ]:
pd.DataFrame(s**2 / (iris_x_scaled.shape[0]), s_eigen[index_eigen])

Pierwsze wiersze czynników głównych (ang. First rows of principal components)

In [ ]:
pca_svd.head(), pca_eigen[index_eigen].head()

### PCA wykres na podstawie 2 pierwszych komponentow

Prosze pamietac ze informacja o odmianach iris-ow (y) nie byla wykorzystana przy obliczaniu PCA

In [ ]:
pca_svd.plot(
    x = 0,
    y = 1, 
    color = pd.Series(iris_set["target"]).map({0: "b", 1: "r", 2: "y"}), 
    kind = "scatter"
)

Iloraz wiariancji pokazuje udział kolejnych komponentow

In [ ]:
s_eigen[index_eigen] / np.sum(s_eigen),\
s**2 / np.sum(s**2)

## Przyklad 2 -- seul1988 -- wyniki dziesiecioboju mezczyzn w Seulu w 1988

In [ ]:
from multidim.datasets import load_seul1988
seul1988 = load_seul1988()
seul1988_copy = seul1988.copy()

Sprawdzamy czy w zbiorze danych sa obserwacje nietypowe - zmienna wynik

In [ ]:
%%stata -d seul1988_copy -force
des

In [ ]:
seul1988.shape, seul1988.dtypes

In [ ]:
%%stata
graph box wynik, title("Wykres pudelkowy")
/*Istnieje jedna potencjalna obserwacja nietypowa.*/

In [ ]:
seul1988["wynik"].plot(kind = "box")

Przygotowujemy zbior danych do analizy -- usuwamy obserwacje potencjalnie nietypowa oraz  przemnazamy wyniki uzyskane w konkurencjach biegowych przez -1, tak aby najnizsza wartosc oznaczala najgorszy wynik

In [ ]:
%%stata -qui
drop if wynik < 6000
gen bieg100_1 = bieg100 * (-1)
gen bieg400_1 = bieg400 * (-1)
gen plotki_1 = plotki * (-1)
gen bieg1500_1 = bieg1500 * (-1)

replace bieg100 = bieg100_1
replace bieg400 = bieg400_1
replace bieg1500 = bieg1500_1
replace plotki = plotki_1
drop bieg100_1 bieg400_1 plotki_1 bieg1500_1

In [ ]:
seul1988 = seul1988.query("wynik >= 6000")
seul1988["bieg100"] = seul1988.bieg100 * -1
seul1988["bieg400"] = seul1988.bieg400 * -1
seul1988["bieg1500"] = seul1988.bieg1500 * -1
seul1988["plotki"] = seul1988.plotki * -1

In [ ]:
%%stata
/*Przeprowadzamy analize glownych skladowych. Domyslnie wykorzystywana jest macierz korelacji*/
corr bieg100-bieg1500
summarize bieg100-bieg1500

In [ ]:
%%stata
/*Przeprowadzamy analize glownych skladowych. Domyslnie wykorzystywana jest macierz korelacji*/
pca bieg100-bieg1500

/*Tylko pierwsze dwie glowne skladowe maja wartosci wlasne wieksze od 1 (co jest rownoznaczne z wyjasniona wariancja wieksza niz srednia) i wyjasniaja ponad
60% calkowitej wariancji.*/

/*Interpretacja dwoch pierwszych skladowych:
pierwsza -- mierzy osiagniety wynik (wszystkie wspolczynniki dodatnie)
druga -- ma wysokie wartosci dla konkurencji zwiazanych z rzucaniem i sila (kula, dysk, oszczep) i
duze ujemne dla wytrzymalosciowych (biegi na 400 i 1500 metrow)*/

/*Okreslenie liczby skladowych glownych, ktore dobrze opisuja wariancje wyjsciowych zmiennych*/

In [ ]:
cols_x = ['bieg100', 'skok_w_dal', 'rzut_kula', 'skok_wzwyz', 
          'bieg400', 'plotki', 'rzut_dysk', 'tyczka', 'oszczep', 'bieg1500']
seul1988_x = seul1988[cols_x]
centred_seul1988_x = pd.DataFrame(seul1988_x - np.mean(seul1988_x), columns=cols_x)
scaled_seul1988_x = pd.DataFrame(seul1988_x - np.mean(seul1988_x), columns=cols_x) / np.std(seul1988_x)

In [ ]:
# seul1988_x.corr()
# seul1988_x.describe().T

In [ ]:
# https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/decomposition/_pca.py
pca = PCA(svd_solver = 'full')
# SVD
# X = U * S * Vt
U, s, Vt = np.linalg.svd(scaled_seul1988_x, full_matrices = False)

Główne składowe (ang. Principal components)

In [ ]:
# principal components
# X_principals = X * V = U * S * Vt * V = U * S
pca_components = pca.fit_transform(scaled_seul1988_x)
np.abs(np.round(pca_components, 2)), np.abs(np.round(np.matmul(scaled_seul1988_x,  Vt.T), 2))
pca.explained_variance_ratio_, (s**2 / np.sum(s**2))

In [ ]:
# rotation matrix
# np.abs needed
np.abs(np.round(Vt.T, 3)), np.abs(np.round(pca.components_.T, 3))
pd.DataFrame(Vt.T)

In [ ]:
%stata predict comp1-comp2

In [ ]:
%%stata
/*Wykres osypiska*/
screeplot, mean /*Wykres kolejnych wartosci wlasnych; "mean" - zostaje naniesiona prosta odpowiadajaca sredniej z wartosci wlasnych*/
screeplot, ci /*Dodatkowo zostaje nalozony przedzial ufnosci*/

In [ ]:
pd.Series(s**2 / seul1988_x.shape[0]).plot()

In [ ]:
%%stata
/*Wykres wspólczynników dla poszczegolnych zmiennych - przydatne przy interpretacji skladowych glownych*/
loadingplot /*Tylko dla dwóch pierwszych wektorów wlasnych*/

In [ ]:
loadings = pd.DataFrame(Vt.T * s / np.sqrt(seul1988_x.shape[0] - 1))
ax = loadings.plot.scatter(0,1)
cols = seul1988_x.columns.tolist()
for i in range(loadings.shape[0]):
    ax.text(x = loadings.iloc[i,0], y = loadings.iloc[i,1], s = cols[i])

Wykres dwoch pierwszych czynników głównych
First two prinicipal components vs not known during extraction "wynik"

In [ ]:
pd.DataFrame(pca_components).plot.scatter(0, 1, c = seul1988["wynik"], colormap = "winter")

In [ ]:
#%%stata
#mvtest normality  bieg100-bieg1500, stats(all)
#pca bieg100-bieg1500, cov vce(normal)
#testparm bieg100-bieg1500, equal eq(Comp1)
#
#/*niestety, nie jest spelnione zalozenie o wielowymiarowym rozkladzie normalnym*/
#
#/*Dalej analizujemy tylko dwie pierwsze glowne skladowe. */

In [ ]:
#%%stata
#/*Jeszcze wyznaczmy macierz korelacji*/
#pwcorr bieg100-bieg1500 wynik comp2 comp1
#sum bieg100-bieg1500 wynik comp2 comp1

In [ ]:
%%stata
//Weryfikacja poprawności przeprowadzonej analizy głównych składowych (PCA)

/*Aby sprawdzic, czy wystarczajaco dobrze odtworzylismy zmiennosc wykorzystamy "reszty" -
roznice pomiedzy zaobserwowanymi korelacjami a tymi odtworzonymi za pomoca tylko kilku pierwszych skladowych*/

pca  bieg100 - bieg1500, components(3) /*Rozwiazanie skladajace sie z 3 pierwszych skladowych glownych*/
estat residual, fit /*Macierz "resztowa" oraz macierz korelacji odtworzona za pomoca 3 pierwszych skladowych*/
corr bieg100 - bieg1500

In [ ]:
cor_original = np.corrcoef(scaled_seul1988_x.T)
# set to zero not used principal components
s3 = np.concatenate([s[:3], [0] * 7]) 
# Eigen -> C = V * (S**2/ (n-1)) * Vt
cor_pca3 = Vt.T @ (np.diag(s3 ** 2) / (seul1988.shape[0] - 1)) @ Vt
pd.DataFrame(cor_original).round(3),\
pd.DataFrame(cor_pca3).round(3),\
pd.DataFrame(cor_original - cor_pca3).round(3)

In [ ]:
%%stata
/*Sprawdzenie, czy zmienne wejsciowe sa ze soba skorelowane.
Czyli sprawdzamy czy ma sens w ogole przeprwowadzenie analizy skladowych glownych*/

/*1. Pierwszy sposob to analiza R^2*/
estat smc
reg  bieg100 skok_w_dal - bieg1500  /*sprawdzenie, skad sie biora te wartosci*/

//WNIOSEK: "Skok wzwyz" wykazuje najmniejsza zaleznosc z pozostalymi konkurencjami

In [ ]:
%%stata
/*2. Drugi sposób:  "anti-image correlation" (minus korelacja czastkowa).*/

/*Korelacja czastkowa pokazuje "czysta" zaleznosc miedzy dwoma zmiennymi, traktujac pozostale jako stale.
Dazymy do uzyskania malej korelacji czastkowej!
Jesli wiele tych korelacji jest relatywnie duzych, to zaleznosc pomiedzy niektorymi zmiennymi nie zalezy od poziomu pozostalych zmiennych.
Tym samym moze byc trudno uzyskac wlasciwe rozwiazanie malego wymiaru. */

pca  bieg100 - bieg1500
estat anti, nocov /*"anti-image correlation" (minus korelacja czastkowa)*/
pcorr  bieg100  skok_w_dal - bieg1500 /*Korelacje czastkowe zmiennej bieg100 z pozostalymi*/

In [ ]:
%%stata
/*3. Trzeci sposób: statystyka adekwatnosci proby Kaiser-Meyer-Olkin.*/
// pierwsza linia kodu  to rezultat uruchamiania Stata przez Jupytera, w Stata ponowne szacowanie 
// analizy skladowych glownych jest zbędne
qui pca  bieg100 - bieg1500
estat kmo

/*Bez  zmiennej "skok_wzwyz"*/
pca bieg100 - kula bieg400 - bieg1500
estat kmo /*Wielkosc statystyki ulegla nieznacznej poprawie*/

/*Wykres wspolczynnikow dla poszczegolnych zmiennych -- przydatne przy interpretacji skladowych glownych*/
pca  bieg100 - bieg1500
loadingplot, comp(3) combined /*3 pierwsze skladowe glowne*/

pca bieg100 - kula bieg400 - bieg1500
loadingplot, comp(3) combined /*bez "skoku wzwyz"*/

## Przykład 3 - Regresja Liniowa

In [ ]:
X = seul1988_x.copy()
X["const"] = 1
y = seul1988["wynik"]
ols = sm.OLS(y, X)
ols_result = ols.fit()
print(ols_result.summary())

In [ ]:
# VIF
pd.DataFrame({
    "variable": seul1988_x.columns, 
    "VIF": [variance_inflation_factor(seul1988_x, i) for i in range(seul1988_x.shape[1])]
})

In [ ]:
x_skok = centred_seul1988_x[['skok_w_dal', 'skok_wzwyz', 'tyczka']]
x_bieg = centred_seul1988_x[['bieg100','bieg400', 'bieg1500', 'plotki']]
x_atlet = centred_seul1988_x[['rzut_kula','rzut_dysk', 'oszczep']]

svd_skok = np.linalg.svd(x_skok, full_matrices = False)
svd_bieg = np.linalg.svd(x_bieg, full_matrices = False)
svd_atlet = np.linalg.svd(x_atlet, full_matrices = False)

print(svd_skok[1]**2/np.sum(svd_skok[1]**2))
print(svd_bieg[1]**2/np.sum(svd_bieg[1]**2))
print(svd_atlet[1]**2/np.sum(svd_atlet[1]**2))

pca_skok = x_skok @ svd_skok[2].T
pca_bieg = x_bieg @ svd_bieg[2].T
pca_atlet = x_atlet @ svd_atlet[2].T

In [ ]:
svd_skok[2].T[:, 0],\
svd_bieg[2].T[:, 0],\
svd_atlet[2].T[:, 0]

In [ ]:
# We do not now the sense (zwrot)
X = pd.concat([pca_skok.iloc[:,0]*-1, pca_bieg.iloc[:,0], pca_atlet.iloc[:,0]*-1], axis = 1)
X.columns = ["skok", "bieg", "atlet"]
print(pd.DataFrame({"variable": X.columns, "VIF": [variance_inflation_factor(X, i) for i in range(X.shape[1])]}))
X["const"] = 1
y = seul1988["wynik"]
ols = sm.OLS(y, X)
ols_result = ols.fit()
print(ols_result.summary())

In [ ]:
# Interpretacja?

## Przyklad 4 -- Objawy depresji (Dane z Diagnoza Spoleczna 2011) PCA

1. Wczytaj zbior danych "depresja.dta"
2. Przeprowadz analize glownych skladowych.
3. Utworz wykres osypiska Ile powinismy wyroznic glownych skladowych?
4. Przeprowadz jeszcze raz analize, ale tylko dla ustalonej w punkcie 3 liczby glownych skladowych. Zapisz glowne skladowe oraz wyjsciowe zmienne w pliku.
5. Zinterpretuj glowne skladowe. Interpretacje poprzyj macierza korelacji.
6. Sprawdz, czy zmienne wejsciowe sa ze soba skorelowane (zweryfikuj, czy ma sens w ogole przeprwowadzenie analizy skladowych glownych).
    Uzyj: R^2, "anti-image correlation" oraz statystyki adekwatnosci Kaisera-Meyera-Olkina.


In [ ]:
from multidim.datasets import load_depresja
depresja = load_depresja()
depresja_copy = depresja.copy()

In [ ]:
%%stata -d depresja_copy -force
des